In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [31]:

a=agent.actor_local.fc1.weight.data.cpu().numpy()
b=agent.actor_local.fc2.weight.data.cpu().numpy()
c=agent.actor_local.fc3.weight.data.cpu().numpy()
#d=agent.actor_local.fc4.weight.data.cpu().numpy()
np.save("actor_local_fc1", a, allow_pickle=True, fix_imports=True)
np.save("actor_local_fc2", b, allow_pickle=True, fix_imports=True)
np.save("actor_local_fc3", c, allow_pickle=True, fix_imports=True)
#np.save("actor_local_fc4", d, allow_pickle=True, fix_imports=True)


In [33]:
a.shape

(400, 3)

### agent definition

In [26]:
import numpy as np
import random
import copy
from collections import namedtuple, deque
import matplotlib.pyplot as plt
#from model import Actor, Critic

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size, random_seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state, in our case the states are the characteristics of our respons: settling time, error, overshoot...
            action_size (int): dimension of each action: the actions here are to modify kp, kd, ki (3x1)
            random_seed (int): random seed
        """
        self.state_size = state_size # 3
        self.action_size = action_size 
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def act(self, state, add_noise=True): 
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)


### actor and critic definition

In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
    

    
    


### main()

In [4]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

evolution_tr = []
evolution_ov = []
evolution_err = []

tr_desire=1 
ov_desire=1.05
err_desire=0
done=0
objectif=np.array([np.array(tr_desire),np.array(ov_desire),np.array(err_desire)])


def calcul(kp,kd,ki):
# prend en entree des numpy array et converti float--> matlab--> float to numpy array
    kp=kp.item()
    ki=ki.item()
    kd=kd.item()
    res=eng.controleur(kp,kd,ki,nargout=3)
    tr=np.array(res[0])
    ov=np.array(res[1])
    err=np.array(res[2])
                
    evolution_tr.append(tr)
    evolution_ov.append(ov)
    evolution_err.append(err)
    
    return np.array([tr,ov,err])

def recomp(objectif,state):
    tr_desire=objectif[0]
    ov_desire=objectif[1]
    err_desire=objectif[2]
    kp=state[0]
    ki=state[1]
    kd=state[2]
    reponse=calcul(kp,kd,ki)
    tr=reponse[0]
    ov=reponse[1]
    err=reponse[2]
    reward=-abs(tr_desire-tr)-abs(ov_desire-ov)-abs(err_desire-err)
    
    if tr_desire>tr and ov_desire>ov and err<0.005:
        done=1
    else:
        done=0
    return reward,done


agent = Agent(state_size=3, action_size=3, random_seed=2)

In [21]:
evolution_kp = []
evolution_kp.append(1)
evolution_kp

[1]

In [ ]:
evolution_kp = []
evolution_kd = []
evolution_ki = []


def ddpg(n_episodes=2000, max_t=300, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        # convertir les gains en np
        kp=np.array(5.0)
        ki=np.array(5.0)
        kd=np.array(5.0)
        
        state=np.array([kp,ki,kd])
        
        # reset notre agent
        agent.reset()
        # reset le score
        score = 0
        for t in range(max_t):
           
            action = agent.act(state)# jai un probleme avec action
            
            print('\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n action1: {}      action2: {}      action3: {} \n      kp: {}           kd: {}            ki: {} '.format(action[0],action[1],action[2],state[0],state[1],state[2]))
           
            
            kp=state[0]+action[0]
            ki=state[1]+action[1]
            kd=state[2]+action[2]
            
            evolution_kp.append(kp)
            evolution_kd.append(kd)
            evolution_ki.append(ki)
            
            next_state=np.array([kp,ki,kd])
            reward,done=recomp(objectif,state)

            
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.title('Score')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_kp)+1), evolution_kp)
plt.ylabel(' Kp')
plt.xlabel('Episode #')
plt.title('Evolution kp')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_kd)+1), evolution_kd)
plt.ylabel('Evolution Kd')
plt.xlabel('Episode #')
plt.title('Evolution kd')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_ki)+1), evolution_ki)
plt.ylabel('Evolution Ki')
plt.xlabel('Episode #')
plt.title('Evolution ki')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_ov)+1), evolution_ov)
plt.ylabel('overshoot')
plt.xlabel('Episode #')
plt.title('Evolution overshoot')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_tr)+1), evolution_tr)
plt.ylabel('temps de reponse')
plt.xlabel('Episode #')
plt.title('Evolution du temps de reponse')
plt.show()

fig = plt.figure()
ax = fig.add_subplot()
plt.plot(np.arange(1, len(evolution_err)+1), evolution_err)
plt.ylabel('Evolution de l''erreur')
plt.xlabel('Episode #')
plt.title('Evolution de l''erreur')
plt.show()


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
ki

In [ ]:

agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
for t in range(200):
    action = agent.act(state, add_noise=False)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()

In [13]:
print('\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n action1: {}      action2: {}      action3: {} \n---------------------------------------------------\n      kp: {}           kd: {}            ki: {} \n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'.format(5,10,11,0,3,4))


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 action1: 5      action2: 10      action3: 11 
---------------------------------------------------
      kp: 0           kd: 3            ki: 4 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
